In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 450kB 7.2MB/s 
     |████████████████████████████████| 870kB 22.7MB/s 
     |████████████████████████████████| 1.0MB 16.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5-cp36-none-any.whl size=64942 sha256=ea2dbf208bbc88508aecaa412fe7d37fc4e92c8f281a8c4c023ee47573511868
  Stored in directory: /root/.cache/pip/wheels/b4/ce/39/5bbda8ac34eb52df8c6531382ca077773fbfcbfb6386e5d66c
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=38f4fcf23bbfddc7f6a1eea7f40968352aebb44feab4c6077cf49549890b37bb
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sentence-transformers sacremoses


In [2]:
!git clone https://github.com/LucaBassanese/Tesi

Cloning into 'Tesi'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 176 (delta 19), reused 27 (delta 10), pack-reused 129
Receiving objects: 100% (176/176), 304.41 MiB | 11.16 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Checking out files: 100% (80/80), done.


# Creazione frasi

In [0]:
import pandas as pd
import re
from itertools import repeat
import random

In [0]:
df = pd.read_excel('/content/Tesi/data/abstract.xlsx')

In [0]:
df = df[df['Abstract inglese'].notnull()]
df = df.drop_duplicates(['Abstract inglese'])
abstracts = list(df['Abstract inglese'])
abstracts = [re.split('©|\(C\)|\(c\)', ab)[0] for ab in abstracts ]
abstracts_fra = [abstract.split('.') for abstract in abstracts]
abstracts_fra =[[sentence.strip() for sentence in abstract if sentence.strip() != ""] for abstract in abstracts_fra]

In [0]:
prima = []
seconda = []

In [0]:
lunghezza = [len(par) for par in abstracts_fra]
indice_bool = [a > 2 for a in lunghezza]

In [0]:
abstracts_fin = []
for i in range(len(abstracts_fra)):
  if indice_bool[i]:
    abstracts_fin.append(abstracts_fra[i])

In [0]:
for a in abstracts_fin:
  casuale = abstracts_fin.copy()
  casuale.remove(a)
  for i in range(len(a)-1):
    prima.extend(repeat(a[i], 2))
    seconda.append(a[i+1])
    seconda.append(casuale[random.randint(0, len(abstracts_fin)-5)][random.randint(0,2)])
    


In [0]:
etichette = ['seguente', 'indipendente'] * int(len(prima)/2)

In [0]:
etichette = [eti + '\n' for eti in etichette ]

In [0]:
prima = [pr + ' \n' for pr in prima]

In [0]:
seconda = [sec + ' \n' for sec in seconda]

In [16]:
len(seconda)

9822

#Applicazione SBERT

In [0]:
from typing import Union, List


class InputExample:
    """
    Structure for one input example with texts, the label and a unique id
    """
    def __init__(self, guid: str, texts: List[str], label: Union[int, float]):
        """
        Creates one InputExample with the given texts, guid and label
        str.strip() is called on both texts.
        :param guid
            id for the example
        :param texts
            the texts for the example
        :param label
            the label for the example
        """
        self.guid = guid
        self.texts = [text.strip() for text in texts]
        self.label = label

In [0]:
import csv
import gzip
import os





def get_examples( filename, max_examples=0):
  """
  data_splits specified which data split to use (train, dev, test).
  Expects that self.dataset_folder contains the files s1.$data_split.gz,  s2.$data_split.gz,
  labels.$data_split.gz, e.g., for the train split, s1.train.gz, s2.train.gz, labels.train.gz
  """
  #s1 = gzip.open(os.path.join(self.dataset_folder, 's1.' + filename),
    #              mode="rt", encoding="utf-8").readlines()
  #s2 = gzip.open(os.path.join(self.dataset_folder, 's2.' + filename),
    #              mode="rt", encoding="utf-8").readlines()
  #labels = gzip.open(os.path.join(self.dataset_folder, 'labels.' + filename),
    #                 mode="rt", encoding="utf-8").readlines()

  examples = []
  id = 0
  for sentence_a, sentence_b, label in zip(prima, seconda, etichette):
      guid = "%s-%d" % (filename, id)
      id += 1
      examples.append(InputExample(guid=guid, texts=[sentence_a, sentence_b], label=map_label(label)))

      if 0 < max_examples <= len(examples):
          break

  return examples

#@staticmethod
def get_labels():
  return {"seguente": 0, "indipendente": 1}

def get_num_labels():
    return len(get_labels())

def map_label( label):
        return get_labels()[label.strip().lower()]

In [0]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime

In [0]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:25<00:00, 15.9MB/s]


In [0]:
train_data = SentencesDataset(get_examples('miosbert'), model=model)


In [0]:
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=32)

In [0]:
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=2)

In [0]:
sts_reader = STSDataReader('/content/Tesi/data/stsbenchmark', normalize_scores=True)
dev_data = SentencesDataset(examples=sts_reader.get_examples('sts-dev.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=32)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=1,
          evaluation_steps=10,
          warmup_steps=100,
          output_path= '/content/sbert')

In [0]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings1 = model.encode(sentences)

In [0]:
sentence_embeddings1 = model.encode(sentences)

In [0]:
sentence_embeddings

In [0]:
model4 = SentenceTransformer('bert-base-nli-mean-tokens')

In [0]:
sentence_embeddings4 = model.encode(sentences)

In [0]:
np.array_equal(sentence_embeddings4, sentence_embeddings1)

True

In [0]:
model.save('/content/salvataggio')

In [0]:
model2 = SentenceTransformer('/content/salvataggio')

In [0]:
model3 =  SentenceTransformer('/content/sbert')

In [0]:
sentence_embeddings3 = model3.encode(sentences)

In [0]:
np.array_equal(sentence_embeddings1, sentence_embeddings2)

True

In [0]:
sentence_embeddings2 = model2.encode(sentences)

In [0]:
import numpy as np

np.array_equal(sentence_embeddings1,sentence_embeddings2)

True

In [0]:
sentence_embeddings1

In [0]:
sentence_embeddings2

In [0]:
prima[500:510]
# Controllo che prima siano duplicati
for i in range(int(len(prima)/2-3)):
  assert prima[i*2] == prima[i*2+1] 
  # print(i)
  # Controllare che è seconda == prima per i pari e diverso per i dispari
  # assert prima[i*2+2] == seconda[i*2] or " " == seconda[i*2]

  # Controllare che i dispari son diversi
  assert prima[i*2+1] != seconda[i*2]

In [0]:
from typing import Union, List


class InputExample:
    """
    Structure for one input example with texts, the label and a unique id
    """
    def __init__(self, guid: str, texts: List[str], label: Union[int, float]):
        """
        Creates one InputExample with the given texts, guid and label
        str.strip() is called on both texts.
        :param guid
            id for the example
        :param texts
            the texts for the example
        :param label
            the label for the example
        """
        self.guid = guid
        self.texts = [text.strip() for text in texts]
        self.label = label

In [0]:
import csv
import gzip
import os


class NLIDataReader(object):
    """
    Reads in the Stanford NLI dataset and the MultiGenre NLI dataset
    """
    def __init__(self, dataset_folder):
        self.dataset_folder = dataset_folder

    def get_examples(self, filename, max_examples=0):
        """
        data_splits specified which data split to use (train, dev, test).
        Expects that self.dataset_folder contains the files s1.$data_split.gz,  s2.$data_split.gz,
        labels.$data_split.gz, e.g., for the train split, s1.train.gz, s2.train.gz, labels.train.gz
        """
        s1 = gzip.open(os.path.join(self.dataset_folder, 's1.' + filename),
                       mode="rt", encoding="utf-8").readlines()
        s2 = gzip.open(os.path.join(self.dataset_folder, 's2.' + filename),
                       mode="rt", encoding="utf-8").readlines()
        labels = gzip.open(os.path.join(self.dataset_folder, 'labels.' + filename),
                           mode="rt", encoding="utf-8").readlines()

        examples = []
        id = 0
        for sentence_a, sentence_b, label in zip(s1, s2, labels):
            guid = "%s-%d" % (filename, id)
            id += 1
            examples.append(InputExample(guid=guid, texts=[sentence_a, sentence_b], label=self.map_label(label)))

            if 0 < max_examples <= len(examples):
                break

        return examples

    @staticmethod
    def get_labels():
        return {"contradiction": 0, "entailment": 1, "neutral": 2}

    def get_num_labels(self):
        return len(self.get_labels())

    def map_label(self, label):
        return self.get_labels()[label.strip().lower()]

In [0]:
filename = 'train.gz'
s1 = gzip.open(os.path.join('/content/Tesi/data/AllNLI', 's1.' + filename),
                       mode="rt", encoding="utf-8").readlines()
s2 = gzip.open(os.path.join('/content/Tesi/data/AllNLI', 's2.' + filename),
                       mode="rt", encoding="utf-8").readlines()
labels = gzip.open(os.path.join('/content/Tesi/data/AllNLI', 'labels.' + filename),
                           mode="rt", encoding="utf-8").readlines()


In [0]:
labels

In [0]:
etichette = [eti + '\n' for eti in etichette ]

In [0]:
labels

In [0]:
s2

In [0]:
nli_reader = NLIDataReader('/content/Tesi/data/AllNLI')

In [0]:
a = nli_reader.get_examples('train.gz')

In [0]:
nli_reader.map_label('entailment')


1

In [0]:
"""
This example loads the pre-trained bert-base-nli-mean-tokens models from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime


#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout


In [0]:

# Read the dataset
model_name = 'bert-base-nli-mean-tokens'
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
sts_reader = STSDataReader('Tesi/data/stsbenchmark', normalize_scores=True)

# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_data = SentencesDataset(sts_reader.get_examples('sts-train.csv'), model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

In [0]:
sts_reader.get_examples('sts-train.csv')

In [0]:
train_data

# Keyword sbert ed autoencoder

In [0]:
import pandas as pd
import re
from itertools import repeat
import random
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime
from scipy import spatial
from itertools import combinations 

In [0]:
matriciona_key = pd.read_excel('/content/Tesi/data/matriciona_key.xlsx', index_col= 0)

In [0]:
parole = pd.read_csv('/content/Tesi/data/keyword.txt', header = None, names= ['parole'])

In [0]:
ind_mod = matriciona_key.index
ind_mod = list(ind_mod)

In [0]:
model = SentenceTransformer(r'/content/drive/My Drive/Modelli/sbert')

In [0]:
comb = combinations(list(parole.parole), 2)
sims = []
v1 = []
v2 = []
for i in list(comb):
  v1.append(i[0])
  v2.append(i[1])


In [0]:
a = model.encode(v1)
b = model.encode(v2)

In [0]:
len(v2)

435

In [0]:
spatial.distance.cosine()

In [0]:
sims = []
for i in range(len(v1)):
  sim = round(1-spatial.distance.cosine(a[i], b[i]), 3)
  sims.append(sim)

In [0]:
matriciona_key = matriciona_key.append(pd.Series(sims, index=matriciona_key.columns ), ignore_index= True)
ind_mod.append('sbert')

## Universal auto encoder

In [0]:
%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn

In [0]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [0]:
# Reduce logging output.
logging.set_verbosity(logging.ERROR)
uni_v1 = embed(v1)
uni_v2 = embed(v2)

In [0]:
a = uni_v1[2]

In [0]:
sims = []
for i in range(len(v1)):
  sim = round(1-spatial.distance.cosine(uni_v1[i], uni_v2[i]), 3)
  sims.append(sim)

In [0]:
matriciona_key = matriciona_key.append(pd.Series(sims, index=matriciona_key.columns ), ignore_index= True)
ind_mod.append('Universal_sent_enc')

In [0]:
matriciona_key.index = ind_mod

In [0]:
matriciona_key

In [0]:
matriciona_key.to_excel('matriciona_key.xlsx')

# Sentence embedding sui titoli

In [0]:
!git clone https://github.com/LucaBassanese/Tesi

Cloning into 'Tesi'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 162 (delta 13), reused 17 (delta 8), pack-reused 129
Receiving objects: 100% (162/162), 289.80 MiB | 33.37 MiB/s, done.
Resolving deltas: 100% (43/43), done.
Checking out files: 100% (76/76), done.


In [0]:
frasi = pd.read_csv('/content/Tesi/data/frasi.txt', header = None, names= ['parole'])

In [0]:
model = SentenceTransformer(r'/content/drive/My Drive/Modelli/sbert')

In [0]:
a = frasi.parole.values

In [0]:
b = model.encode(a)

In [0]:
z = model.encode(a[15])

In [0]:
sentence_bertamb = pd.DataFrame(b)


In [0]:
sentence_bertamb.to_csv('SBERT.csv', index= False)